In [1]:
%pylab inline
from htmd import *

Populating the interactive namespace from numpy and matplotlib


Using Anaconda Cloud api site https://api.anaconda.org


New HTMD version (1.0.23) is available. You are currently on (1.0.16). Use 'conda update htmd' to update to the new version.


In [2]:
mol = Molecule("pub.htmd.org/CXCL12-confAnalysis/1x14/structure.pdb")

In [35]:
molare = Molecule("pub.htmd.org/CXCL12-confAnalysis/1x14/structure.pdb")

In [3]:
mol.read('pub.htmd.org/CXCL12-confAnalysis/1x14/traj.xtc')

In [4]:
mol.wrap('protein')

In [5]:
mol.align('protein')


In [6]:
from htmd.molecule.util import maxDistance
D = maxDistance(mol, 'all')
maxdistance = D*1.1
maxdistance


56.227686833697874

In [7]:
mol.filter('name OH2 and x^2+y^2+z^2<'+str(round(maxdistance)**2))

In [8]:
mol.coords.shape

(7066, 3, 2000)

In [39]:
mol.moveBy([maxdistance, maxdistance, maxdistance])
molare.moveBy([maxdistance, maxdistance, maxdistance])


In [10]:
heymax=np.max(mol.coords,axis=0)
homax = np.max(heymax,axis=1)
print(homax,homax.shape)

[ 108.59597015  108.58717346  315.99526978] (3,)


In [11]:
heymin=np.min(mol.coords,axis=0)
homin = np.min(heymin,axis=1)
print(homin,homin.shape)

[-212.74359131  -79.66153717    3.40529919] (3,)


It seems the filtering does not work as intended, as the extreme points are not excluded

In [12]:
np.round((homax-homin)*1.1)

array([ 353.,  207.,  344.], dtype=float32)

In [13]:
cubedistance=int(round(maxdistance*2))

In [14]:
grid = np.zeros([cubedistance,cubedistance,cubedistance])
grid.shape

(112, 112, 112)

Okay, so iterate for every atom, and round the coordinates to use them to sum +1 in the zero array, and then divide by the total of atoms of Oxigen, and then transform to energies. You can use the maxdistance to create the cube and so on. 

In [15]:
import itertools

for atom, frame in itertools.product(*map(range, (mol.coords.shape[0], mol.coords.shape[2]))):
    coord_list = [mol.coords[atom][0][frame], mol.coords[atom][1][frame], mol.coords[atom][2][frame]]
    if max(coord_list) > 112:
        print(coord_list[0],coord_list[1], coord_list[2])
    if min(coord_list) < 0:
        print(coord_list[0],coord_list[1], coord_list[2])
    else:
        continue
print("Done!")



-212.744 -79.6615 315.995
-212.744 -79.6615 315.995
Done!


In [15]:
import itertools

for atom, frame in itertools.product(*map(range, (mol.coords.shape[0], mol.coords.shape[2]))):
    coord_list = [mol.coords[atom][0][frame], mol.coords[atom][1][frame], mol.coords[atom][2][frame]]
    if max(coord_list) > 112:
        continue
    if min(coord_list) < 0:
        continue
    else:
        grid[int(round(coord_list[0]))][int(round(coord_list[1]))][int(round(coord_list[2]))] += 1
print("Done!")

Done!


In [16]:
grid[75][62][62]/7066

0.010897254457967733

In [28]:
grid_div = grid/7066
grid_div.shape

(112, 112, 112)

In order to calculate the free energy, i'll use the Boltzman constant with kcal/mol/K as units.

In [29]:
for x,y,z in itertools.product(*map(range, (grid_div.shape[0], grid_div.shape[1],grid_div.shape[2]))):
    prob = grid_div[x][y][z]
    if prob == 0:
        continue
    else:
        grid_div[x][y][z] = (np.log(grid_div[x][y][z])) * 0.001987191 * 298 * -1 
        
grid_div[75][62][62]




2.6762193395389895

In [19]:
for x,y,z in itertools.product(*map(range, (grid_div.shape[0], grid_div.shape[1],grid_div.shape[2]))):
    prob = grid_div[x][y][z]
    if isinstance(prob,str):
        print("ERROR")
    else:
        continue

In [32]:
from htmd.molecule.util import writeVoxels
min_vec= np.array([0,0,0])
max_vec=np.array([112,112,112])
res_vec=np.array([1,1,1])
writeVoxels(grid_div,'isosurf.cube', min_vec, max_vec, res_vec )